In [1]:
%%capture
import contrails_utils

In [2]:
import gc
import os

import numpy as np
import pandas as pd

import yaml
from pathlib import Path

In [3]:
### Kaggle environment variable
INTERACTIVE = os.getenv("KAGGLE_KERNEL_RUN_TYPE") == "Interactive"
RERUN = os.getenv("KAGGLE_IS_COMPETITION_RERUN") is not None
print(f"Interactive {INTERACTIVE}, RERUN {RERUN}")

Interactive False, RERUN False


In [4]:
# Path("pseudolabel/train").mkdir(exist_ok=True, parents=True)

In [5]:
!cp -r /kaggle/input/pseudolabelling/pseudolabel pseudolabel

In [6]:
# MODEL_PATH = "/kaggle/working/models/"
MODEL_PATH = Path("/kaggle/input/training-with-4-folds/models")
with open(MODEL_PATH.parent/ "config.yaml", "r") as file_obj:
    config = yaml.safe_load(file_obj)

In [7]:
config["batch_size"] = 128
config["num_workers"] = 1
threshold = 0.35
N_TIMES_BEFORE = 4
DO_LABELING = "validation"
data_path = Path('/kaggle/input/google-research-identify-contrails-reduce-global-warming')

### pseudo labeling on unlabeled sequence (4 before & 3 after the labbeled one)

In [8]:
%%time
if DO_LABELING and not RERUN:
#     paths_model = list(Path("/kaggle/input/training-with-4-folds/models").glob("*.ckpt"))
#     paths_model = list(Path("/kaggle/input/training-with-4-folds/models").glob("*661*.ckpt"))
    paths_model = [Path("/kaggle/input/training-with-4-folds/models/model-f0-val_dice=0.6616.ckpt")]
    save_empty_contrails = False
    train_df = pd.DataFrame([{'record_id': int(path_train.name), 'path':path_train} for path_train in (data_path/DO_LABELING).iterdir()])
    for seq_num in [1]:
        train_df["sequence_num"] = seq_num
        preds, models_len = contrails_utils.PyLModel.predict_with_models(train_df,models=paths_model, config=config)
        for record_id in train_df.record_id:
            predicted_mask_with_threshold = contrails_utils.PyLModel.stack_prediction_mask_threshold(preds, record_id, threshold, models_len)
            save_path = Path(f"pseudolabel/{DO_LABELING}/{record_id}/sequence_{seq_num}")
            if save_empty_contrails or predicted_mask_with_threshold.sum()>0:
                save_path.mkdir(exist_ok=True, parents=True)
                np.save(save_path/"label.npy",predicted_mask_with_threshold)
            

Predict with model: model-f0-val_dice=0.6616.ckpt
CPU times: user 23.6 s, sys: 4.09 s, total: 27.7 s
Wall time: 2min 20s
